In [27]:
import findspark
findspark.init()
import pandas as pd
from pyspark import SparkConf, SparkContext
import os
import re
import time
import datetime
import sys
import json

conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)

In [28]:
K = 160
cluster_file_location = "word2vec_clustering_cluster_limit_word2vec2_cluster_kmeans{}_total.csv".format(K)


In [29]:
if not os.path.isdir('./data_{}_cluster'.format(K)):
    os.mkdir('./data_{}_cluster'.format(K))
    
save_path = './data_{}_cluster/post_feature_{}.json'.format(K, K)
clustered_post_path = './data_{}_cluster/clustered_post_{}.json'.format(K, K)

In [30]:
def convert_to_stp(date):
    date = date.split('/')
    date[0] = "{0:0>2}".format(int(date[0]))
    date[1] = "{0:0>2}".format(int(date[1]))
    date = date[0] + '/' + date[1] + '/20' + date[2]
    return time.mktime(datetime.datetime.strptime(date, "%m/%d/%Y").timetuple())


if not os.path.isfile('./wordpress_combine.csv'):
    !gsutil -m cp gs://digital-trend-data-unzipped/user_clicks_combine/wordpress/wordpress_combine.csv ./wordpress_combine.csv	
posts = sc.textFile('./wordpress_combine.csv')\
                .map(lambda line: line.split(','))\
                .filter(lambda line: line[0] != '0')\

post_tsp = posts.map(lambda line: (line[0], convert_to_stp(line[1])))

print(posts.take(3))
print(post_tsp.take(10))
print(post_tsp.count())

post_tsp_dict = post_tsp.collectAsMap()
print(post_tsp_dict.get('1002075'))

[['1002075', '4/3/19', 'fashion|travel|Travel|Travel|packing|luggage|apparel', '5 Best Travel Clothes Brands You Can Get Away with Not Washing', 'www.themanual.com/travel/best-travel-clothes-brands'], ['56635', '4/2/19', 'celular|Telefonía celular|Mobile|celular|como-grabar-pantalla-del-celular|quicktime|lonely-screen|apowersoft', 'Te mostramos cómo grabar la pantalla del iPhone fácilmente', 'es.digitaltrends.com/celular/como-grabar-pantalla-celular'], ['396726', '4/2/19', 'computing|buying-guides|Computing|Computing|hdd|ssd|sshd', 'SSD vs. HDD', 'www.digitaltrends.com/computing/ssd-vs-hdd']]
[('1002075', 1554249600.0), ('56635', 1554163200.0), ('396726', 1554163200.0), ('2199706', 1554163200.0), ('2310644', 1554076800.0), ('2312038', 1554076800.0), ('81655', 1553990400.0), ('2312093', 1553990400.0), ('979013', 1553904000.0), ('1315200', 1553904000.0)]
176359
1554249600.0


In [31]:

if not os.path.isfile(cluster_file_location):
    !gsutil -m cp gs://digital-trend-data-unzipped/word2vec_clustering/cluster_limit/{cluster_file_location}	 ./{cluster_file_location}
post_feature_cluster = sc.textFile(cluster_file_location)\
                    .map(lambda info: info.split(','))\
                    .map(lambda info: (info[0], tuple(info[1:34])))\
                    .filter(lambda info: info[0] != 'post_id')
print(post_feature_cluster.take(2))


[('1421442', ('0.098892563', '0.102089292', '-0.318239731', '-0.208830575', '-0.068282231', '-0.255948234', '0.162419267', '0.272043177', '-0.001014436', '-0.065607232', '0.064096039', '-0.008541029', '0.342712751', '0.191695274', '-0.012319952', '0.208137059', '-0.046524898', '0.105477104', '-0.040210796', '0.193598983', '-0.037529704', '0.185677126', '-0.012594694', '0.103968483', '-0.041162812', '-0.006860914', '0.124144584', '-0.024743152', '0.011455852', '0.49490109', '-0.189881916', '0.245100581', '0')), ('2197914', ('0.07727495', '0.162456005', '-0.359939277', '-0.121451972', '-0.111216197', '-0.266144592', '0.198612034', '0.245898306', '0.086982125', '-0.02679752', '0.059231659', '0.076752615', '0.361376139', '0.206717229', '0.107092298', '0.095726339', '-0.075782881', '0.13601805', '-0.051357334', '0.182648898', '-0.141830681', '0.129822855', '0.004728177', '0.098957763', '0.0213878', '-0.107089783', '0.15885546', '0.020287991', '0.115594646', '0.472407426', '-0.17365275', '0.

In [32]:
def to_float(T):
    res = []
    for t in T:
        res.append(float(t))
    return res

def get_timestamp(date):
    return post_feature_cluster.value.get(date)

post_feature = post_feature_cluster.mapValues(tuple)\
                .map(lambda info: (info[0], list( tuple(to_float(info[1])) + (post_tsp_dict.get(info[0]), ))))
print(post_feature.count())
print(post_feature.take(4))


176359
[('1421442', [0.098892563, 0.102089292, -0.318239731, -0.208830575, -0.068282231, -0.255948234, 0.162419267, 0.272043177, -0.001014436, -0.065607232, 0.064096039, -0.008541029, 0.342712751, 0.191695274, -0.012319952, 0.208137059, -0.046524898, 0.105477104, -0.040210796, 0.193598983, -0.037529704, 0.185677126, -0.012594694, 0.103968483, -0.041162812, -0.006860914, 0.124144584, -0.024743152, 0.011455852, 0.49490109, -0.189881916, 0.245100581, 0.0, 1534204800.0]), ('2197914', [0.07727495, 0.162456005, -0.359939277, -0.121451972, -0.111216197, -0.266144592, 0.198612034, 0.245898306, 0.086982125, -0.02679752, 0.059231659, 0.076752615, 0.361376139, 0.206717229, 0.107092298, 0.095726339, -0.075782881, 0.13601805, -0.051357334, 0.182648898, -0.141830681, 0.129822855, 0.004728177, 0.098957763, 0.0213878, -0.107089783, 0.15885546, 0.020287991, 0.115594646, 0.472407426, -0.17365275, 0.131488158, 0.0, 1531872000.0]), ('412141', [0.095356972, 0.13117271, -0.37405098, -0.264683779, -0.0606890

In [39]:
post_feature_reduce = post_feature.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 2)
print(post_feature_reduce.take(10))

[('41476', 3), ('42738', 3), ('51905', 3), ('2.20E+06', 695), ('2.31E+06', 716), ('1.00E+06', 552), ('5.73E+05', 43), ('2.30E+06', 962), ('1.40E+06', 769), ('1.25E+06', 726)]


In [33]:
with open(save_path, 'w') as outfile:  
    json.dump(post_feature.collectAsMap(), outfile)

In [34]:
# extract post feature: (id, vector)
with open(save_path, 'r') as inputfile:  
    post_feature_dict = json.load(inputfile)

print(post_feature_dict.get('1002075'))
print(len(post_feature_dict.keys()))

[0.281270855, 0.109975921, 0.124338905, -0.221134741, 0.107404406, -0.061329004, 0.336072625, -0.103838169, -0.080011655, 0.095860341, 0.102366746, -0.173454508, 0.221690306, 0.124662899, -0.004446651, -0.202376869, 0.024064676, 0.230010116, 0.094039662, 0.237077338, -0.111818731, 0.281354555, 0.077846064, 0.062100764, -0.235334693, 0.069466248, 0.142621747, -0.017744301, 0.162133874, 0.455479974, -0.049766763, 0.079899357, 7.0, 1554249600.0]
61507


In [18]:
# visualize post in one cluster
# post id and keywords
wordpress_file_location = "./wordpress_combine.csv"
if not os.path.isfile(wordpress_file_location):
    !gsutil -m cp gs://digital-trend-data-unzipped/wordpress/wordpress_combine.csv	 {wordpress_file_location}
wordpress = sc.textFile(wordpress_file_location).map(lambda x: x.split(',')).map(lambda x: (x[0], x[3])).collectAsMap()
print(wordpress.get('1002075'))


5 Best Travel Clothes Brands You Can Get Away with Not Washing


In [19]:
joined_cluster = post_feature.map(lambda x: (int(x[1][-2]), x[0])).groupByKey().mapValues(list).collectAsMap()

cluster_Keys = joined_cluster.keys()
cluster_info = {}
for key in cluster_Keys:
    cluster_info[key] = {}
    one_cluster = cluster_info[key]
    for post in joined_cluster.get(key):
        one_cluster[post] = {}
        post_info = one_cluster[post]
        post_info['title'] = wordpress.get(post)
        post_info['feature'] = post_feature_dict.get(post)
        
with open(clustered_post_path, 'w') as outfile:  
    json.dump(cluster_info, outfile)